## ClimaCoreMakie

[Makie.jl](https://makie.juliaplots.org/stable/) is a data visualization ecosystem for the Julia programming language, with high performance and extensibility. 

Makie.jl can use several backends. Two backends of interest are: CairoMakie (for 2D plots) and GLMakie (for 3D plots)

In [ ]:
using IntervalSets

import ClimaCore
import ClimaCoreMakie: ClimaCoreMakie, Makie

using GLMakie

### 2D non-hybrid plane

In [ ]:
domain = ClimaCore.Domains.RectangleDomain(
    ClimaCore.Geometry.XPoint(0) .. ClimaCore.Geometry.XPoint(2π),
    ClimaCore.Geometry.YPoint(0) .. ClimaCore.Geometry.YPoint(2π),
    x1periodic = true,
    x2periodic = true,
)

n1, n2 = 2, 2
Nq = 4
mesh = ClimaCore.Meshes.RectilinearMesh(domain, n1, n2)
grid_topology = ClimaCore.Topologies.Topology2D(mesh)
quad = ClimaCore.Spaces.Quadratures.ClosedUniform{Nq}()
space = ClimaCore.Spaces.SpectralElementSpace2D(grid_topology, quad)
coords = ClimaCore.Fields.coordinate_field(space)

sinxy = map(coords) do coord
    cos(coord.x + coord.y)
end

# Let's plot this scalar-valued field
plane_fig = ClimaCoreMakie.viz(sinxy)

In [ ]:
# And let's save it
plane_fig_filename = joinpath(".", "Makie_2D_rectangle.png")
GLMakie.save(plane_fig_filename, plane_fig)

### 2D sphere

In [ ]:
R = 6.37122e6

domain = ClimaCore.Domains.SphereDomain(R)
mesh = ClimaCore.Meshes.EquiangularCubedSphere(domain, 6)
grid_topology = ClimaCore.Topologies.Topology2D(mesh)
quad = ClimaCore.Spaces.Quadratures.GLL{5}()
space = ClimaCore.Spaces.SpectralElementSpace2D(grid_topology, quad)
coords = ClimaCore.Fields.coordinate_field(space)

u = map(coords) do coord
    u0 = 20.0
    α0 = 45.0
    ϕ = coord.lat
    λ = coord.long

    uu = u0 * (cosd(α0) * cosd(ϕ) + sind(α0) * cosd(λ) * sind(ϕ))
    uv = -u0 * sind(α0) * sind(λ)
    ClimaCore.Geometry.UVVector(uu, uv)
end

# Let us plot the zonal component of the velocity
u_field_fig = ClimaCoreMakie.viz(u.components.data.:1)

In [ ]:
# Let us save it
u_field_fig_filename = joinpath("./", "Makie_2D_cubed_sphere.png")
GLMakie.save(u_field_fig_filename, u_field_fig)